In [10]:
import functions as f
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [11]:
sat_files = f.read_satellite_data_9D(num_years=2)
dat = xr.open_dataset(sat_files[2011][0]).squeeze()
sat_proj = ccrs.NorthPolarStereo()
sss = dat['sss']
print(sss)

<xarray.DataArray 'sss' (y: 432, x: 432)>
[186624 values with dtype=float32]
Coordinates:
    time     datetime64[ns] 2011-07-17T11:48:00
  * y        (y) float32 -5.388e+03 -5.362e+03 ... 5.362e+03 5.388e+03
  * x        (x) float32 -5.388e+03 -5.362e+03 ... 5.362e+03 5.388e+03
    lat      (y, x) float32 ...
    lon      (y, x) float32 ...
Attributes:
    units:          1
    standard_name:  sea_surface_salinity
    long_name:      Sea Surface Salinity
    description:    Sea Surface Salinity [psu]
    grid_mapping:   crs
    valid_min:      0.0
    valid_max:      50.0


In [12]:
f.plot_sss_sat(sss)

NameError: name 'plt' is not defined